In [1]:
pip install pandas sqlalchemy ipython-sql


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade prettytable

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade ipython-sql


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import sqlite3
import pandas as pd


In [5]:
import prettytable
prettytable.__dict__['DEFAULT'] = prettytable.PLAIN_COLUMNS


C:\Users\Zaki Bouaoudia\AppData\Local\Temp\ipykernel_10208\1122509105.py:2: DeprecationWarning: the 'PLAIN_COLUMNS' constant is deprecated, use the 'TableStyle' enum instead
  prettytable.__dict__['DEFAULT'] = prettytable.PLAIN_COLUMNS


In [6]:
# List of CSV file paths
csv_files = {
    "Appearances": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project - Cursor AI\Football Database\appearances.csv",
    "Games": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project - Cursor AI\Football Database\games.csv",
    "Leagues": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project - Cursor AI\Football Database\leagues.csv",
    "Players": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project - Cursor AI\Football Database\players.csv",
    "Shots": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project - Cursor AI\Football Database\shots.csv",
    "Teams": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project - Cursor AI\Football Database\teams.csv",
    "TeamStats": r"C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project - Cursor AI\Football Database\teamstats.csv",
}

In [7]:
# Create SQLite database (file-based)
db_file = "football_database.db"
conn = sqlite3.connect(db_file)  # Use a persistent database file

In [8]:
# Load each CSV into a separate table
for table_name, file_path in csv_files.items():
    try:
        # Specify a fallback encoding
        df = pd.read_csv(file_path, encoding='latin1')
        df.to_sql(table_name, conn, index=False, if_exists='replace')
        print(f"Loaded {file_path} into table {table_name}")
    except UnicodeDecodeError as e:
        print(f"Error loading {file_path}: {e}")


Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project - Cursor AI\Football Database\appearances.csv into table Appearances
Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project - Cursor AI\Football Database\games.csv into table Games
Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project - Cursor AI\Football Database\leagues.csv into table Leagues
Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project - Cursor AI\Football Database\players.csv into table Players
Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project - Cursor AI\Football Database\shots.csv into table Shots
Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project - Cursor AI\Football Database\teams.csv into table Teams
Loaded C:\Users\Zaki Bouaoudia\OneDrive\09_Programming\SQL Project\EU Football SQL Project -

In [9]:
# Verify tables are loaded
print("Tables in the database:")
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)

Tables in the database:
          name
0  Appearances
1        Games
2      Leagues
3      Players
4        Shots
5        Teams
6    TeamStats


In [10]:
# Load SQL magic
%load_ext sql

In [11]:
# Configure SQL magic display settings
%config SqlMagic.feedback = False
%config SqlMagic.autopandas = True  # This converts results to pandas DataFrames
%config SqlMagic.displaywidth = 1000  # Controls width

# Add pandas display options to control both width and length
pd.set_option('display.max_rows', 20)     # Limits number of rows
pd.set_option('display.max_columns', 10)  # Limits number of columns
pd.set_option('display.width', 1000)      # Controls overall display width
pd.set_option('display.max_colwidth', 100) # Controls maximum column width

c:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3284: UserWarning: Config option `displaywidth` not recognized by `SqlMagic`.  Did you mean one of: `displaycon, displaylimit`?
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [12]:
%sql sqlite:///football_database.db  


In [13]:
%%sql
SELECT COUNT(*)

FROM Leagues





 * sqlite:///football_database.db


,COUNT(*)
0,5


In [30]:

%%sql
# Get all teams IDs that played in the Premier League
SELECT DISTINCT homeTeamID as id
FROM games
WHERE leagueID in (SELECT leagueID
                  FROM leagues
                  WHERE leagueID = 1)
                  



 * sqlite:///football_database.db


,id
0,89
1,73
2,72
3,75
4,79
...,...
26,228
27,229
28,227
29,238


In [33]:
%%sql
--Display every team name in the premier league
WITH premierLeagueIDs AS (SELECT DISTINCT homeTeamID as id
FROM games
WHERE leagueID in (SELECT leagueID
                  FROM leagues
                  WHERE leagueID = 1))

SELECT t.teamID, t.name
FROM teams t
WHERE t.teamID in (SELECT id
                   FROM premierLeagueIDs);



 * sqlite:///football_database.db


,teamID,name
0,71,Aston Villa
1,72,Everton
2,74,Southampton
3,75,Leicester
4,76,West Bromwich Albion
...,...,...
26,227,Cardiff
27,228,Fulham
28,229,Wolverhampton Wanderers
29,238,Sheffield United


In [32]:
%%sql
--Calculate how many goals Arsenal scored at home by season
SELECT 
    season, 
    SUM(homegoals) as Total_Home_Goals
        
FROM games g
WHERE g.homeTeamID = 83
GROUP BY season
ORDER BY season;

 * sqlite:///football_database.db


,season,Total_Home_Goals
0,2014,41
1,2015,31
2,2016,39
3,2017,54
4,2018,42
5,2019,36
6,2020,24


In [21]:
%%sql
#Calculate how many goals Arsenal scored at home vs away from the 2014 to 2020 season
SELECT 
    season, 
    SUM(CASE WHEN homeTeamID = 83 THEN homegoals ELSE 0 END) AS Total_Home_Goals,
    SUM(CASE WHEN awayTeamID = 83 THEN awaygoals ELSE 0 END) AS Total_Away_goals
FROM games
WHERE (homeTeamID = 83
    OR awayTeamID = 83)
GROUP BY season
ORDER BY season;

 * sqlite:///football_database.db


,season,Total_Home_Goals,Total_Away_goals
0,2014,41,30
1,2015,31,34
2,2016,39,38
3,2017,54,20
4,2018,42,31
5,2019,36,20
6,2020,24,31


In [22]:
- this query is wrong redo it

In [67]:
%%sql
--Calculate the most goals scored by a team in the Premier league 2020 indicating the team name, the side, the opponent name, the date and the maximum goals
WITH Max_Goals_Table AS (
    SELECT MAX(homeGoals) AS Max_Goals, homeTeamID AS Team_ID, 'Home' AS Side, date, awayTeamID AS Opponent_ID
    FROM games g
    WHERE g.leagueID =1 AND g.season = '2020'

   UNION ALL

   SELECT MAX(awayGoals) AS Max_Goals, awayTeamID AS Team_ID, 'Away' AS Side, date, homeTeamID AS Opponent_ID
   FROM games g
   WHERE g.leagueID =1 AND g.season = '2020')

SELECT mg.Max_Goals AS 'Maximum Goals in a Game',
       (CASE WHEN mg.Side = 'Home' THEN home.name
       WHEN mg.Side = 'Away' THEN away.name END) AS Team_Name,
       mg.Side, 
       (CASE WHEN mg.Side = 'Home' THEN (SELECT away.name FROM teams away WHERE away.teamID = mg.Opponent_ID)
       WHEN mg.side = 'Away' THEN (SELECT home.name FROM teams home WHERE home.teamID = mg.Opponent_ID) END) AS Opponent_Name,
       strftime('%d-%m-%Y', mg.date) AS Date
FROM Max_Goals_Table mg
LEFT JOIN teams home ON home.teamID = mg.team_ID
LEFT JOIN teams away ON away.teamID = mg.team_ID
WHERE mg.Max_Goals = (SELECT MAX(Max_Goals) FROM Max_Goals_Table)


 * sqlite:///football_database.db


,Maximum Goals in a Game,Team_Name,Side,Opponent_Name,Date
0,9,Manchester United,Home,Southampton,02-02-2021


In [59]:
%%sql


 * sqlite:///football_database.db


,Max_Goals,Team_ID,Side,date,Opponent_ID
0,9,89,Home,2021-02-02 20:15:00,74
1,7,87,Away,2020-12-19 12:30:00,78


In [25]:
%%sql
/* Shows all winners and losers in 2020 Premier League matches with: */
SELECT  g.season, 
        g.date, 
        (SELECT t.name
         FROM teams t
         WHERE t.teamID = g.homeTeamID
        ) AS Home_Team,
        (SELECT t.name
         FROM teams t
         WHERE t.teamID = g.awayTeamID
        ) AS Away_Team, 
        (CASE WHEN homeGoals > awayGoals THEN "Home Team Wins"
        WHEN homeGoals < awayGoals THEN "Away Team Wins"
        ELSE 'Tie' END) As Winning_Side
FROM games g
WHERE leagueID = 1 and season = '2020'
ORDER BY date DESC;



 * sqlite:///football_database.db


,season,date,Home_Team,Away_Team,Winning_Side
0,2020,2021-05-23 15:00:00,Arsenal,Brighton,Home Team Wins
1,2020,2021-05-23 15:00:00,Aston Villa,Chelsea,Home Team Wins
2,2020,2021-05-23 15:00:00,Fulham,Newcastle United,Away Team Wins
3,2020,2021-05-23 15:00:00,Leeds,West Bromwich Albion,Home Team Wins
4,2020,2021-05-23 15:00:00,Leicester,Tottenham,Away Team Wins
...,...,...,...,...,...
375,2020,2020-09-13 13:00:00,West Bromwich Albion,Leicester,Away Team Wins
376,2020,2020-09-12 19:00:00,West Ham,Newcastle United,Away Team Wins
377,2020,2020-09-12 16:30:00,Liverpool,Leeds,Home Team Wins
378,2020,2020-09-12 14:00:00,Crystal Palace,Southampton,Home Team Wins


In [26]:
#Same query as above writen differently

In [28]:
%%sql
/* This query shows all Premier League matches with their dates and results,
 joining with the teams table to get team names and determining the winner*/
SELECT g.season, 
    g.date, 
    home.name AS HomeTeamName,
    away.name AS AwayTeamName,
    CASE 
        WHEN g.homeGoals > g.awayGoals THEN 'Home Team Wins'
        WHEN g.homeGoals < g.awayGoals THEN 'Away Team Wins'
        ELSE 'TIE' 
    END AS Winning_Side
FROM games g
INNER JOIN teams home ON g.homeTeamID = home.teamID
INNER JOIN teams away ON g.awayTeamID = away.teamID
WHERE g.leagueID = 1
ORDER BY g.date DESC;

 * sqlite:///football_database.db


,season,date,HomeTeamName,AwayTeamName,Winning_Side
0,2020,2021-05-23 15:00:00,Arsenal,Brighton,Home Team Wins
1,2020,2021-05-23 15:00:00,Aston Villa,Chelsea,Home Team Wins
2,2020,2021-05-23 15:00:00,Fulham,Newcastle United,Away Team Wins
3,2020,2021-05-23 15:00:00,Leeds,West Bromwich Albion,Home Team Wins
4,2020,2021-05-23 15:00:00,Leicester,Tottenham,Away Team Wins
...,...,...,...,...,...
2655,2014,2014-08-16 15:00:00,Queens Park Rangers,Hull,Away Team Wins
2656,2014,2014-08-16 15:00:00,Stoke,Aston Villa,Away Team Wins
2657,2014,2014-08-16 15:00:00,West Bromwich Albion,Sunderland,TIE
2658,2014,2014-08-16 15:00:00,West Ham,Tottenham,Away Team Wins


In [ ]:
#Rank teams according to the number of games won over all seasons for premier league teams

#first query is the previous one with a CTE

#Second Query calculate the HOme_games one for each team

In [33]:
%%sql

WITH premMatches AS (SELECT  g.season, 
        g.date, 
        (SELECT t.name
         FROM teams t
         WHERE t.teamID = g.homeTeamID
        ) AS Home_Team,
        (SELECT t.name
         FROM teams t
         WHERE t.teamID = g.awayTeamID
        ) AS Away_Team, 
        (CASE WHEN homeGoals > awayGoals THEN "Home Team Wins"
        WHEN homeGoals < awayGoals THEN "Away Team Wins"
        ELSE 'Tie' END) As Winning_Side
FROM games g
WHERE leagueID = 1 
ORDER BY date DESC)

SELECT TEAM, SUM(Wins) AS "Total Wins"
FROM (

SELECT Home_Team AS Team, COUNT(*) As Wins
FROM premMatches 
WHERE Winning_Side = "Home Team Wins"
GROUP BY Home_Team

UNION ALL

SELECT Away_Team AS TEAM, COUNT(*) As Wins
FROM premMatches 
WHERE Winning_Side = "Away Team Wins"
GROUP BY Away_Team
) AS Combined_Wins
GROUP BY TEAM
ORDER BY "Total Wins" DESC
LIMIT 6;


 * sqlite:///football_database.db
Done.


Team,Total Wins
Manchester City,183
Liverpool,159
Chelsea,149
Tottenham,144
Manchester United,140
Arsenal,137


In [29]:
#Find the reiging champions in the permier league

SELECT homeTeamID, 
LAG(,1) OVer()
FROM games
WHERE leagueID = 1



SyntaxError: invalid syntax (218226232.py, line 3)

In [104]:

%%sql
--New query to test out the pushing to github
SELECT *
FROM games
LIMIT 10;


 * sqlite:///football_database.db


gameID,leagueID,season,date,homeTeamID,awayTeamID,homeGoals,awayGoals,homeProbability,drawProbability,awayProbability,homeGoalsHalfTime,awayGoalsHalfTime,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,PSCH,PSCD,PSCA
81,1,2015,2015-08-08 15:45:00,89,82,1,0,0.2843,0.3999,0.3158,1,0,1.65,4.0,6.0,1.65,4.0,5.5,1.65,3.6,5.1,1.65,4.09,5.9,1.62,3.6,6.0,1.67,4.0,5.75,1.64,4.07,6.04
82,1,2015,2015-08-08 18:00:00,73,71,0,1,0.3574,0.35,0.2926,0,0,2.0,3.6,4.0,2.0,3.3,3.7,2.1,3.3,3.3,1.95,3.65,4.27,1.91,3.5,4.0,2.0,3.5,4.2,1.82,3.88,4.7
83,1,2015,2015-08-08 18:00:00,72,90,2,2,0.2988,0.4337,0.2675,0,1,1.7,3.9,5.5,1.7,3.5,5.0,1.7,3.6,4.7,1.7,3.95,5.62,1.73,3.5,5.0,1.73,3.9,5.4,1.75,3.76,5.44
84,1,2015,2015-08-08 18:00:00,75,77,4,2,0.6422,0.2057,0.1521,3,0,1.95,3.5,4.33,2.0,3.3,3.75,2.0,3.3,3.6,1.99,3.48,4.34,2.0,3.1,2.7,2.0,3.4,4.33,1.79,3.74,5.1
85,1,2015,2015-08-08 18:00:00,79,78,1,3,0.1461,0.2159,0.638,0,1,2.55,3.3,3.0,2.6,3.2,2.7,2.4,3.2,2.85,2.52,3.35,3.08,2.6,3.1,2.88,2.6,3.25,3.0,2.46,3.39,3.14
86,1,2015,2015-08-08 20:30:00,80,84,2,2,0.0304,0.0924,0.8772,2,1,1.36,5.0,11.0,1.4,4.75,9.0,1.33,4.8,8.3,1.39,4.92,10.39,1.4,4.0,10.0,1.4,5.0,9.5,1.37,5.04,10.88
87,1,2015,2015-08-09 16:30:00,86,74,2,2,0.4386,0.3165,0.2449,1,1,2.88,3.3,2.7,2.8,3.1,2.75,2.65,3.3,2.5,2.88,3.33,2.69,2.7,3.1,2.7,2.88,3.25,2.7,3.09,3.28,2.55
88,1,2015,2015-08-09 16:30:00,83,81,0,2,0.5751,0.2826,0.1423,0,1,1.29,6.0,12.0,1.28,5.75,10.5,1.33,4.8,8.3,1.31,5.75,12.0,1.3,5.0,11.0,1.3,5.75,12.0,1.24,6.75,15.0
89,1,2015,2015-08-09 19:00:00,85,87,0,1,0.2482,0.5454,0.2064,0,0,3.4,3.4,2.3,3.2,3.4,2.3,2.9,3.3,2.3,3.48,3.46,2.25,3.3,3.1,2.3,3.4,3.4,2.3,3.89,3.51,2.09
90,1,2015,2015-08-10 23:00:00,76,88,0,3,0.0615,0.1698,0.7687,0,2,5.75,4.0,1.67,4.75,4.0,1.65,5.1,3.6,1.65,5.75,3.98,1.68,5.5,3.5,1.7,5.5,4.0,1.7,6.46,4.08,1.61


In [ ]:
#automate queries across all tables
for table_name in csv_files.keys():
    query = f"SELECT COUNT(*) AS row_count FROM {table_name};"
    result = pd.read_sql_query(query, conn)
    print(f"Table {table_name} has {result['row_count'][0]} rows.")

In [ ]:
conn = sqlite3.connect('multi_table_database.db')
# Your tables are already saved if using this connection
conn.close()
